In [2]:
import cv2
import numpy as np
import dlib
from math import hypot

In [ ]:
cap = cv2.VideoCapture(0)
rednose = cv2.imread("rednose.png")
antlers = cv2.imread("antlers.png")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
print(rednose.shape)
print(antlers.shape)

while True:
    ret, frame = cap.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(frame)
    for face in faces:
        landmarks = predictor(gray_frame, face)
        topnose = (landmarks.part(29).x, landmarks.part(29).y)
        centernose = (landmarks.part(30).x, landmarks.part(30).y)
        leftnose = (landmarks.part(31).x, landmarks.part(31).y)
        rightnose = (landmarks.part(35).x, landmarks.part(35).y)
        midbrow = (landmarks.part(34).x, landmarks.part(23).y)
        botnose = (landmarks.part(34).x, landmarks.part(34).y)
        forehead = int(hypot(midbrow[0] - botnose[0],midbrow[1] - botnose[1]))
        tophead = (landmarks.part(34).x, int(landmarks.part(23).y + forehead))
        righthead = (landmarks.part(1).x, landmarks.part(1).y)
        lefthead = (landmarks.part(17).x, landmarks.part(17).y)
        midantler = (landmarks.part(34).x, (int(landmarks.part(23).y + forehead) + forehead))
        antlerwidth = int((hypot(lefthead[0] - righthead[0], lefthead[1] - righthead[1]))*5)
        antlerheight = int(antlerwidth*0.6)
        nosewidth = int(hypot(leftnose[0] - rightnose[0],leftnose[1] - rightnose[1]))
        noseheight = nosewidth
        toplefta = (int(midantler[0] - antlerwidth/2), int(midantler[1] - antlerheight/2))
        botrighta = (int(midantler[0] + antlerwidth/2), int(midantler[1] + antlerheight/2))
        topleft = (int(centernose[0] - nosewidth/2), int(centernose[1] - noseheight/2))
        botright = (int(centernose[0] + nosewidth/2), int(centernose[1] + noseheight/2))
        nosearea = frame[topleft[1]:topleft[1] + noseheight, topleft[0]:topleft[0] + nosewidth]
        antlerarea = frame[toplefta[1]:toplefta[1] + antlerheight, toplefta[0]:toplefta[0] + antlerwidth]
        nosered = cv2.resize(rednose, (nosewidth, noseheight))
        antlernew = cv2.resize(antlers, (antlerwidth, antlerheight))
        noseredgray = cv2.cvtColor(nosered, cv2.COLOR_BGR2GRAY)
        antlernewgray = cv2.cvtColor(antlernew, cv2.COLOR_BGR2GRAY)
        _, nosemask = cv2.threshold(noseredgray, 25, 255, cv2.THRESH_BINARY_INV)
        _, antlermask = cv2.threshold(antlernewgray, 25, 255, cv2.THRESH_BINARY_INV)
        nonose = cv2.bitwise_and(nosearea, nosearea, mask = nosemask)
        noantler = cv2.bitwise_and(antlerarea, antlerarea, mask = antlermask)
        finalnose = cv2.add(nonose, nosered)
        finalantler = cv2.add(noantler, antlernew)
        frame[topleft[1]:topleft[1] + noseheight, topleft[0]:topleft[0] + nosewidth] = finalnose
        frame[toplefta[1]:toplefta[1] + antlerheight, toplefta[0]:toplefta[0] + antlerwidth] = finalantler
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

(98, 100, 3)
(368, 610, 3)
